# **Proyecto La Liga** 
*Gabriela Vega.*

*Juan Albarracin.*

*Ángel Gómez.*

**Sistema de competición:**

La competición se disputa anualmente, consta de un grupo único integrado por **veinte equipos**.  se enfrentan todos contra todos en **dos ocasiones** (una en campo propio y otra en campo contrario). El orden de los encuentros se decide por sorteo antes de empezar la competición.

La clasificación final se establece con arreglo a los puntos totales obtenidos por cada equipo al finalizar el campeonato. Los equipos obtienen **tres puntos por cada partido ganado, un punto por cada empate y ningún punto por los partidos perdidos.**

**Objetivo del Proyecto:**

El objetivo principal de este proyecto es simular la tabla de clasificaciones de la liga para así poder predecir resultados de partidos para la realización de apuestas.

### 1.Importar Datos


Tomamos nuestro dataset y lo exploramos

In [0]:
import numpy as np
import pandas as pd

In [0]:
from google.colab import files
up = files.upload()

Saving FMEL_Dataset.csv to FMEL_Dataset (1).csv


In [0]:
data = pd.read_csv("FMEL_Dataset.csv")

In [0]:
data

,id,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date,timestamp
0,1,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,12/09/1970,21938400
1,2,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,12/09/1970,21938400
2,3,1970-71,1,1,Real Madrid,Valencia,2,0,12/09/1970,21938400
3,4,1970-71,1,1,Celta de Vigo,Sporting de Gijon,2,0,13/09/1970,22024800
4,5,1970-71,1,1,Elche,Granada,1,1,13/09/1970,22024800
...,...,...,...,...,...,...,...,...,...,...
37142,37143,2017-18,2,42,Valladolid,Osasuna,2,0,02/06/2018,1527890400
37143,37144,2017-18,2,42,Oviedo,Huesca,2,1,02/06/2018,1527890400
37144,37145,2017-18,2,42,Alcorcon,Reus,3,0,02/06/2018,1527890400
37145,37146,2017-18,2,42,Numancia,Cultural Leonesa,2,1,02/06/2018,1527890400


## 2.Segmentación de datos

Tomaremos únicamente los equipos de la primera división.

In [0]:
df=data[data.division==1]
df=df[['localTeam','visitorTeam', 'localGoals',	'visitorGoals']]
df

,localTeam,visitorTeam,localGoals,visitorGoals
0,Atletico de Bilbao,Barcelona,1,1
1,Las Palmas,Atletico de Madrid,1,1
2,Real Madrid,Valencia,2,0
3,Celta de Vigo,Sporting de Gijon,2,0
4,Elche,Granada,1,1
...,...,...,...,...
36680,Villarreal,Real Madrid,2,2
36681,Atletico de Bilbao,Espanol,0,1
36682,Barcelona,Real Sociedad,1,0
36683,Valencia,Deportivo,2,1


Como vemos, tenemos 17169 resgistros de partidos de 1a división.

## 3.Funciones de probabilidad

**Función partido**

Halla la probabilidad de victoria, empate y derrota para 2 equipos dados.

In [0]:
def partido(local, visitante):
    Win= data[(data.localTeam==local) & (data.visitorTeam==visitante) & (data.localGoals>data.visitorGoals)]
    empatar = data[(data.localTeam==local) & (data.visitorTeam==visitante) & (data.localGoals==data.visitorGoals)]

    probWin = (Win.shape[0])/(data[(data.localTeam==local) & (data.visitorTeam==visitante)].shape[0]) 
    probEmp = (empatar.shape[0])/(data[(data.localTeam==local) & (data.visitorTeam==visitante)].shape[0])

    probLoss = 1-probWin-probEmp
    
    return probWin, probEmp, probLoss


In [0]:
partido("Barcelona", "Real Madrid")

(0.5918367346938775, 0.24489795918367346, 0.163265306122449)

**Función puntosPartido**

Función para, una vez conocidos las probabilidades de victoria o derrota, pueda determinar mediante numeros aleatorios el resultado del encuentro entre 2 equipos, recibe como parametro el resultado de la función partido.

In [0]:
def puntosPartido(prob):
  i = np.random.random()
  puntosLocal=0;
  puntosVisit=0;
  if i <= prob[0]: %victoria
    puntosLocal=3
    puntosVisit=0
  elif (i>prob[0] and i<(prob[1]+prob[0])): %empate
    puntosLocal=1
    puntosVisit=1
  else:  %derrota
    puntosLocal =0
    puntosVisit=3
    
  return puntosLocal, puntosVisit

In [0]:
puntosPartido(partido("Barcelona", "Real Madrid"))

(1, 1)

## 4.Generar tabla de encuentros

Vamos a generar todos los encuentros que tendrá cada equipo

In [0]:
print (np.sort(df.localTeam.unique()))

['AD Almeria' 'Alaves' 'Albacete' 'Almeria' 'Atletico de Bilbao'
 'Atletico de Madrid' 'Barcelona' 'Betis' 'Burgos' 'CD Malaga' 'Cadiz'
 'Castellon' 'Celta de Vigo' 'Compostela' 'Cordoba' 'Deportivo' 'Eibar'
 'Elche' 'Espanol' 'Extremadura' 'Getafe' 'Gimnastic de Tarragona'
 'Girona' 'Granada' 'Hercules' 'Las Palmas' 'Leganes' 'Levante' 'Lleida'
 'Logrones' 'Malaga' 'Mallorca' 'Merida' 'Murcia' 'Numancia' 'Osasuna'
 'Oviedo' 'Rayo Vallecano' 'Real Burgos' 'Real Madrid' 'Real Sociedad'
 'Recreativo de Huelva' 'Sabadell' 'Salamanca' 'Santander' 'Sevilla'
 'Sporting de Gijon' 'Tenerife' 'Valencia' 'Valladolid' 'Villarreal'
 'Xerez' 'Zaragoza']


In [0]:
df.localTeam.unique().shape

(53,)

Seleccionamos nuestros 20 equipos para que jueguen La Liga (Esta es la temporada 2019-20)

In [0]:
a = np.array(['Barcelona','Real Madrid','Sevilla','Real Sociedad','Getafe','Atletico de Madrid',
              'Valencia','Villarreal','Granada','Atletico de Bilbao','Osasuna','Betis',
              'Levante','Alaves','Valladolid','Eibar','Celta de Vigo','Mallorca',
              'Leganes','Espanol'])

In [0]:
#np.delete(df.localTeam.unique(),0)

## 5.Simulación de La Liga

Simulamos todos los encuentros y por cada uno determinamos el resultado (los puntos para cada equipo)

In [0]:
c=[]
for i in range(len(a)):
  b=np.delete(a,i)
  for j in b:
    d=puntosPartido(partido(a[i],j))
    c.append([a[i],j,d[0],d[1]])

In [0]:
c

[['Barcelona', 'Real Madrid', 3, 0],
 ['Barcelona', 'Sevilla', 3, 0],
 ['Barcelona', 'Real Sociedad', 3, 0],
 ['Barcelona', 'Getafe', 3, 0],
 ['Barcelona', 'Atletico de Madrid', 1, 1],
 ['Barcelona', 'Valencia', 3, 0],
 ['Barcelona', 'Villarreal', 3, 0],
 ['Barcelona', 'Granada', 3, 0],
 ['Barcelona', 'Atletico de Bilbao', 3, 0],
 ['Barcelona', 'Osasuna', 3, 0],
 ['Barcelona', 'Betis', 3, 0],
 ['Barcelona', 'Levante', 3, 0],
 ['Barcelona', 'Alaves', 3, 0],
 ['Barcelona', 'Valladolid', 0, 3],
 ['Barcelona', 'Eibar', 3, 0],
 ['Barcelona', 'Celta de Vigo', 3, 0],
 ['Barcelona', 'Mallorca', 3, 0],
 ['Barcelona', 'Leganes', 3, 0],
 ['Barcelona', 'Espanol', 3, 0],
 ['Real Madrid', 'Barcelona', 3, 0],
 ['Real Madrid', 'Sevilla', 3, 0],
 ['Real Madrid', 'Real Sociedad', 3, 0],
 ['Real Madrid', 'Getafe', 3, 0],
 ['Real Madrid', 'Atletico de Madrid', 1, 1],
 ['Real Madrid', 'Valencia', 3, 0],
 ['Real Madrid', 'Villarreal', 1, 1],
 ['Real Madrid', 'Granada', 3, 0],
 ['Real Madrid', 'Atletico de B

## 6.Resultados Obtenidos

Convertimos los resultados, que son una lista, a numpy array y luego construimos un nuevo dataset con los resultados de toda la Liga ya jugada.

In [0]:
resultados=np.asarray(c)
resultados

array([['Barcelona', 'Real Madrid', '3', '0'],
       ['Barcelona', 'Sevilla', '3', '0'],
       ['Barcelona', 'Real Sociedad', '3', '0'],
       ...,
       ['Espanol', 'Celta de Vigo', '1', '1'],
       ['Espanol', 'Mallorca', '3', '0'],
       ['Espanol', 'Leganes', '3', '0']], dtype='<U18')

Estos son los resultados de todos los partidos jugados

In [0]:
#Ese astype(int) es porque los puntos son strings y toca convertirlos.
laLiga=pd.DataFrame({'Local':resultados[:,0], 'Visitante':resultados[:,1], 'Puntos_Local':resultados[:,2].astype(int), 'Puntos_Visitante':resultados[:,3].astype(int)})
laLiga

,Local,Visitante,Puntos_Local,Puntos_Visitante
0,Barcelona,Real Madrid,3,0
1,Barcelona,Sevilla,3,0
2,Barcelona,Real Sociedad,3,0
3,Barcelona,Getafe,3,0
4,Barcelona,Atletico de Madrid,1,1
...,...,...,...,...
375,Espanol,Valladolid,1,1
376,Espanol,Eibar,0,3
377,Espanol,Celta de Vigo,1,1
378,Espanol,Mallorca,3,0


In [0]:
t=[]
for i in a:
  puntos_totales = laLiga[laLiga.Local==i].Puntos_Local.sum()+laLiga[laLiga.Visitante==i].Puntos_Visitante.sum()
  t.append([i,puntos_totales])

t2=np.asarray(t)
tablaPosiciones=pd.DataFrame({"Equipo":t2[:,0], "Puntos":t2[:,1]})
tablaPosiciones

,Equipo,Puntos
0,Barcelona,77
1,Real Madrid,92
2,Sevilla,46
3,Real Sociedad,56
4,Getafe,32
5,Atletico de Madrid,56
6,Valencia,64
7,Villarreal,59
8,Granada,39
9,Atletico de Bilbao,67


In [0]:
#Ordenado por puntos de mayor a menor
tablaPosiciones.sort_values(by=["Puntos"],ascending=False)
#tablaPosiciones.index

,Equipo,Puntos
1,Real Madrid,92
0,Barcelona,77
9,Atletico de Bilbao,67
6,Valencia,64
11,Betis,63
7,Villarreal,59
3,Real Sociedad,56
5,Atletico de Madrid,56
16,Celta de Vigo,51
2,Sevilla,46
